### Pipeline to label DIMACS road networks with TIGER/Line

In [ ]:
# Purpose: The purpose of this notebook is to attach category of each edge of DIMACS road networks.
# Source: We use TIGER/Line dataset from DIMACS to attach categories to edges
# Approach: 
#     1. We first download target road network from 
#       http://www.diag.uniroma1.it/challenge9/download.shtml, NY for the rest of description.
#     2. Then we download all the TIGER/Line from 
#       http://www.diag.uniroma1.it/challenge9/data/tiger/, and we merge them into USA_ALL.tmp for reference.
#       merge.cpp can be found at "Merging of two or more files" on the same webpage,
#       and compile it to 'merge.out' before running
#     3. We use USA_ALL.tmp as dictionary to guide the labeling process of target road network, and
#       save it as USA-road-l.NY.gr in the format of DIMACS.

In [ ]:
# WorkFlow:
# http://www.diag.uniroma1.it/challenge9/data/tiger/merge.cpp
# - Data preparation(with merge.out)
#   |
#   +- pullFromTigerLine()
#   |
#   +- unzipTigerLine()
#   |
#   +- mergetmps2USAALL()
#   |
#   +: Return: Now we have USA_ALL.tmp
#
# - Label given graph
#   |
#   +: Description: Given graph from Dimacs like NY, 
#      we label it with coor2id_dict and edge2wc_dict,
#      and output to USA-road-l.NY.gr
#   |
#   +- labelGivenGr_with_USA_ALL()
#   |
#   +: Write as: USA-road-l.NY.gr

In [ ]:
# we store this notebook to g_source_dir, and use it as our workspace
g_source_dir = "TIGERLine"

### Data Preparation

In [ ]:
def pullFromTigerLine():
    import os
    import requests
    
    # Base URL
    base_url = "http://www.diag.uniroma1.it/challenge9/data/tiger/"
    
    # List of state abbreviations (50 states + D.C.)
    states = [
        "AK", "AL", "AR", "AZ", "CA", "CO", "CT", "DC", "DE", "FL", "GA",
        "HI", "IA", "ID", "IL", "IN", "KS", "KY", "LA", "MA", "MD", "ME",
        "MI", "MN", "MO", "MS", "MT", "NC", "ND", "NE", "NH", "NJ", "NM",
        "NV", "NY", "OH", "OK", "OR", "PA", "RI", "SC", "SD", "TN", "TX",
        "UT", "VA", "VT", "WA", "WI", "WV", "WY"
    ]
    
    # Directory to save files
    save_dir = g_source_dir
    print("Saving to ", save_dir)
    os.makedirs(save_dir, exist_ok=True)
    
    # Download each state's file
    for state in states:
        file_name = f"{state}.tmp.gz"
        file_url = base_url + file_name
        file_path = os.path.join(save_dir, file_name)
    
        print(f"Downloading {file_name}...")
    
        response = requests.get(file_url, stream=True)
        if response.status_code == 200:
            with open(file_path, "wb") as file:
                for chunk in response.iter_content(chunk_size=8192):
                    file.write(chunk)
            print(f"Saved: {file_path}")
        else:
            print(f"Failed to download: {file_name}")
    
    print("All downloads completed.")

In [ ]:
def unzipTigerLine_Road():
    import os
    import gzip
    import shutil
    
    # Directory containing the downloaded .tmp.gz files
    source_dir = g_source_dir
    
    # Unzipping each file
    for file_name in os.listdir(source_dir):
        if file_name.endswith(".tmp.gz"):  # Check for .tmp.gz files
            gz_path = os.path.join(source_dir, file_name)
            tmp_path = os.path.join(source_dir, file_name[:-3])  # Remove .gz extension
    
            print(f"Unzipping {file_name}...")
    
            with gzip.open(gz_path, 'rb') as gz_file:
                with open(tmp_path, 'wb') as tmp_file:
                    shutil.copyfileobj(gz_file, tmp_file)
    
            print(f"Unzipped: {tmp_path}")
    
    print("All files have been unzipped.")
    

In [ ]:
def mergetmps2USAALL():
    import subprocess
    #Thx to GPT for gerating list of 50 states of US
    states = [
        "AK", "AL", "AR", "AZ", "CA", "CO", "CT", "DC", "DE", "FL", "GA",
        "HI", "IA", "ID", "IL", "IN", "KS", "KY", "LA", "MA", "MD", "ME",
        "MI", "MN", "MO", "MS", "MT", "NC", "ND", "NE", "NH", "NJ", "NM",
        "NV", "NY", "OH", "OK", "OR", "PA", "RI", "SC", "SD", "TN", "TX",
        "UT", "VA", "VT", "WA", "WI", "WV", "WY"
    ]
    ech_str = ""
    for ele in states:
        tmpstr = g_source_dir+ele
        ech_str+=tmpstr+".tmp "
    print(ech_str)
    
    cmd_pre = 'echo "'
    # merge.cpp compiled as merge.out before running
    cmd_post = '" | ./merge.out USA_ALL.tmp'
    
    cmd=""
    cmd+=cmd_pre
    cmd+=ech_str
    cmd+=cmd_post
    print(cmd)
    
    # Run the command
    process = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    
    # Print the output
    print("STDOUT:", process.stdout)
    print("STDERR:", process.stderr)

In [ ]:
def pullGrCo(GivenGrCo_name):
    import requests
    url = "http://www.diag.uniroma1.it/challenge9/data/USA-road-d/USA-road-d."+GivenGrCo_name+".gz"
    filename = "USA-road-d."+GivenGrCo_name+".gr.gz"

    # Send HTTP request
    response = requests.get(url, stream=True)

    # Check if request was successful
    if response.status_code == 200:
        with open(filename, "wb") as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        print(f"Downloaded {filename} successfully.")
    else:
        print(f"Failed to download {filename}. HTTP Status Code: {response.status_code}")

def pullFromDIMACS(GivenGr_Lis):
   tmp_Lis = [] 
   tmp_Lis = GivenGr_Lis
   for name in tmp_Lis:
       pullGrCo(name+".gr") 
       pullGrCo(name+".co") 


In [ ]:
# Data Preparation
# TIGER/Line
pullFromTigerLine()
# Road networks
pullGrCo_Lis = ["NY", "COL", "FLA", "CAL"]
pullFromDIMACS(pullGrCo_Lis)
# gunzip
unzipTigerLine_Road()
# output USA_ALL.tmp
mergetmps2USAALL()

### Label Given Graph

In [1]:
def read_tiger_line_tmpfile(in_file):
    """
    Input:
       @param in_file
       It is a tiger line file. E.g. DC.tmp. Descriptions can be found on dimacs
       http://www.diag.uniroma1.it/challenge9/data/tiger/
    Return:
        @param num_of_nodes
        @param num_of_edges
        @param coor2id_dict
        @param edge2wc_dict
        We return coor2id_dict using (lon, log) as key to ids, 
        and edge2wc_dict of (from_id, to_id) as key to (edge weight, category)
    """
    print("read TIGER/LINE TMP file:", in_file)
    i=0
    num_of_nodes = 0
    num_of_edges = 0 
    flag = 0
    coor2id_dict = {}
    edge2wc_dict = {}
    with open(in_file, "r") as file:
        for line in file:  # Read the file line by line
            if 0 == i:
                num_of_nodes = line
                num_of_nodes = int(num_of_nodes)  
                print("# nodes: ",num_of_nodes)
                i += 1
            elif i<num_of_nodes+1:
                #e.g. 9550 -77036974 38910146
                id, lon, log = line.split()
                id = int(id)
                lon = int(lon)
                log = int(log)
                coor2id_dict[(lon, log)] = id
                # print(id, lon, log)
                i += 1
            elif i==num_of_nodes+1:
                num_of_edges = line
                num_of_edges = int(line)
                print("# of edges: ", num_of_edges)
                i += 1
            # no need to add i as rest of file are edge infor
            else:
                if 0 == flag:
                    src, des = line.split()
                    src = int(src)
                    des = int(des)
                    edge2wc_dict[(src, des)] = (-1.0, -1)
                    flag = 1
                    # print(src, des)
                elif 1==flag:
                    trav_time, distance, category = line.split()
                    trav_time = float(trav_time)
                    distance = float(distance)
                    category = int(category)
                    edge2wc_dict[(src, des)] = (distance, category)
                    flag = 0
                    # print(trav_time, distance, category)
    return num_of_edges, num_of_edges, coor2id_dict, edge2wc_dict 



In [2]:
def readGivenGr(in_file_co, in_file_gr):
    print("read Given Graph: ", in_file_co, " and ", in_file_gr)
    GG_coor2id_dict = {}
    GG_edge2wc_dict = {}
    GG_edgeOrigOrder_Lis = []
    i=0
    with open(in_file_co, "r") as file:
        for line in file:  # Read the file line by line
            if i<7:
                i+=1
                continue
            else:
                v, id, lon, log = line.split()
                id = int(id)
                lon = int(lon)
                log = int(log)
                GG_coor2id_dict[(lon, log)] = id
                # print(v, id, lon, log) 
    i=0
    with open(in_file_gr, "r") as file:
        for line in file:  # Read the file line by line
            if i<7:
                i+=1
                continue
            else:
                a, uid, vid, dist = line.split()
                # print(a, uid, vid, dist)
                uid = int(uid)
                vid = int(vid)
                dist = int(dist)
                GG_edge2wc_dict[(uid, vid)] = (dist, -1)
                GG_edgeOrigOrder_Lis.append((uid, vid))
    return GG_coor2id_dict, GG_edge2wc_dict, GG_edgeOrigOrder_Lis


In [ ]:
def labelGivenGr_with_USA_ALL(GivenGr, l_size, n, m, coor2id_dict, edge2wc_dict):
    '''
    Members:
        GGId2DictId: given NY ids we find correspondind ids in USA_ALL.tmp
        DictId2GGId: reverse
        GG_edgeOrigOrder_Lis: keep the orignal order we read from NY.gr
        GG_edge2wc_dict: updated with edge category 
        key_lis: labels sorted in decending order of frequence
        s and e:
            |-s = e
            |-e = math.floor(s+(len(key_lis) - i -1 )/l_size + 1)
            s is the start pos of new label
            e is the end pos of the new label
            we rename the labels by shrinking label of similar frequncy into one new label
        
    Write:
        we write GG_edge2wc_dict to file in DIMACS format.
    '''
    GG_coor2id_dict, GG_edge2wc_dict, GG_edgeOrigOrder_Lis = \
                readGivenGr("./"+g_source_dir+"/USA-road-d."+GivenGr+".co", "./"+g_source_dir+"/USA-road-d."+GivenGr+".gr")
    # get infor from USA_ALL.tmp dict 
    # node mapping
    GGId2DictId = {}
    DictId2GGId = {}
    for key in GG_coor2id_dict.keys():
        dictId = coor2id_dict[key]
        GGId = GG_coor2id_dict[key]
        GGId2DictId[GGId] = dictId
        DictId2GGId[dictId] = GGId
    cnt = 0
    for key in GG_edge2wc_dict.keys():
        # get dict id of given gr
        (dictId_from, dictId_to) = (GGId2DictId[key[0]], GGId2DictId[key[1]])
        (GG_wei, _) = GG_edge2wc_dict[key]
        # get edge category from dict
        if (dictId_from, dictId_to) in edge2wc_dict.keys():
            (_, dict_cat) = edge2wc_dict[(dictId_from, dictId_to)]
            # update given graph category
            GG_edge2wc_dict[key] = (GG_wei, dict_cat)
        elif (dictId_to, dictId_from) in edge2wc_dict.keys():
            (_, dict_cat) = edge2wc_dict[(dictId_to, dictId_from)]
            GG_edge2wc_dict[key] = (GG_wei, dict_cat)
        else:
            cnt+=1
    print(cnt)
    # rebalance categories into 10 categories as described in VLDB2024
    category_dict = {}
    for ele in GG_edgeOrigOrder_Lis:
        (_, GG_cat) = GG_edge2wc_dict[ele]
        if GG_cat in category_dict.keys():
            category_dict[GG_cat]+=1
        else:
            category_dict[GG_cat] = 0
    print(category_dict)
    key_lis = []
    for key in category_dict.keys():
        key_lis.append((key, category_dict[key]))
    key_lis.sort(reverse=True, key=lambda x:x[1])
    print(key_lis)
    import math
    s = 0
    e = 0
    # l_size = 10
    olde2new_label={}
    print(len(key_lis))
    # we map number segments to l_size segments
    for i in range(l_size):
        s = e
        e = math.floor(s+(len(key_lis) - i -1 )/l_size + 1)
        for j in range(s, e):
           olde2new_label[key_lis[j][0]] = i
    print(olde2new_label)
    # USA-road-l.NY.gr
    out_labeled_file = "./"+g_source_dir+"/USA-road-l."+GivenGr+".gr"
    with open(out_labeled_file, "w") as file:
        file.write("c 9th DIMACS Implementation Challenge: Shortest Paths\n")
        file.write("c http://www.dis.uniroma1.it/~challenge9\n")
        file.write("c TIGER/Line nodes coords for graph "+out_labeled_file+"\n")
        file.write("c\n")
        file.write("p sp co "+str(n)+" "+str(m)+"\n")
        file.write("c graph contains "+str(n)+" nodes and "+str(m)+" arcs\n")
        file.write("c\n")
        for ele in GG_edgeOrigOrder_Lis:
            (GG_from, GG_to) = ele
            (GG_wei, GG_category) = GG_edge2wc_dict[ele]
            # we use ceil()*10 weight to keep same as weight from 
            # http://www.diag.uniroma1.it/challenge9/data/USA-road-d/USA-road-d.NY.gr.gz
            # write relabeled category to file
            file.write("a "+str(GG_from)+" "+str(GG_to)+" "+str(GG_wei)+" "+str(olde2new_label[GG_category])+"\n")
    print("----------")

In [ ]:
TigeLineTmp = "USA_ALL"
n, m, coor2id_dict, edge2wc_dict =  read_tiger_line_tmpfile("./"+g_source_dir+"/"+TigeLineTmp+".tmp")

read TIGER/LINE TMP file: ./TIGERLine/USA_ALL.tmp
# nodes:  24412259
# of edges:  29653988


In [ ]:
GivenGr_Lis = ["NY", "COL", "FLA", "CAL"]
GivenGr_lis=[]
for GivenGr in GivenGr_Lis: 
    labelGivenGr_with_USA_ALL(GivenGr, 10, n, m, coor2id_dict, edge2wc_dict)

read Given Graph:  ./TIGERLine/USA-road-d.NY.co  and  ./TIGERLine/USA-road-d.NY.gr
0
{41: 692519, 15: 10993, 21: 4133, 25: 2895, 31: 13933, 35: 4837, 45: 845, 11: 3461, 44: 5, 42: 17, 12: 33, 37: 1, 38: 3, 43: 37, 48: 11, 13: 25, 16: 79, 46: 1}
[(41, 692519), (31, 13933), (15, 10993), (35, 4837), (21, 4133), (11, 3461), (25, 2895), (45, 845), (16, 79), (43, 37), (12, 33), (13, 25), (42, 17), (48, 11), (44, 5), (38, 3), (37, 1), (46, 1)]
18
{41: 0, 31: 0, 15: 1, 35: 1, 21: 2, 11: 2, 25: 3, 45: 3, 16: 4, 43: 4, 12: 5, 13: 5, 42: 6, 48: 6, 44: 7, 38: 7, 37: 8, 46: 9}
read Given Graph:  ./TIGERLine/USA-road-d.COL.co  and  ./TIGERLine/USA-road-d.COL.gr
0
{41: 987102, 11: 6453, 21: 24579, 31: 36338, 13: 389, 15: 835, 33: 27, 45: 91, 25: 575, 42: 73, 43: 299, 16: 35, 44: 41, 35: 151, 23: 31, 24: 1, 17: 3, 48: 9, 37: 1, 12: 1, 36: 3, 47: 3, 38: 3}
[(41, 987102), (31, 36338), (21, 24579), (11, 6453), (15, 835), (25, 575), (13, 389), (43, 299), (35, 151), (45, 91), (42, 73), (44, 41), (16, 35), 